# Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import ast

In [2]:
movies_metadata_raw = pd.read_csv('data/movies_metadata_raw.csv')
movies_metadata_raw

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


### We need to normalize dataset for exploration.So we create function to convert json string columns to separate datasets and then drop them from movies_metadata

In [3]:
def to_df(df, column_name):
    records = []
    # df.iterrows returns index and row data as iterator
    for _, row in df[['id', column_name]].iterrows():
        if row[column_name] not in (np.NaN, 'nan'):
            data_json = ast.literal_eval(row[column_name])  # converts string to json dictionary
            # We expect list of dictionaries, but some rows are corrupt which contains
            # bool or float data. Checking if value is bool or float then we skip them
            if isinstance(data_json, bool) or isinstance(data_json, float):
                continue
            else:
                for data in data_json:
                    data['movie_id'] = row.id
                    records.append(data)

    return pd.DataFrame(records)


movie_genres = []
for _, row in movies_metadata_raw[['id', 'genres']].iterrows():
    genres_json = ast.literal_eval(row.genres)
    for genre in genres_json:
        genre['movie_id'] = row.id
        genre.pop('id')
        movie_genres.append(genre)

movie_genres = pd.DataFrame(movie_genres)


movie_languages = []
for _, row in movies_metadata_raw[['id', 'spoken_languages']].iterrows():
    if row.spoken_languages not in (np.NaN, 'nan') :
        genres_json = ast.literal_eval(row.spoken_languages)
        for genre in genres_json:
            genre['movie_id'] = row.id
            movie_languages.append(genre)

movie_languages = pd.DataFrame(movie_languages)

records = []
for _, row in movies_metadata_raw[['id', 'production_companies']].iterrows():
    if row.production_companies not in (np.NaN, 'nan'):
        data_json = ast.literal_eval(row.production_companies)
        if isinstance(data_json, list):
            for data in data_json:
                data['movie_id'] = row.id
                records.append(data)

# Creating 'movie_languages' and 'movie_genres' csv files and dropping them from movies_metadata
movie_languages.to_csv('data/movie_languages.csv', index=False)
movie_genres.to_csv('data/movie_genres.csv', index=False)
movies_metadata_raw.drop(['genres', 'spoken_languages'], axis=1, inplace=True)

# Creating 'production_companies' and 'production_countries'csv files and dropping them from movies_metadata
production_companies = to_df(movies_metadata_raw, 'production_companies')
production_companies.to_csv('data/production_companies.csv', index=False)

# Creating 'production_countries' dataframe and csv file
production_countries = to_df(movies_metadata_raw, 'production_countries')
production_countries.to_csv('data/production_countries.csv', index=False)

# Dropping them from movies_metadata
movies_metadata_raw.drop(['production_companies','belongs_to_collection','production_countries'], axis=1, inplace=True)


## Movie genres dataframe

In [4]:
movie_genres = pd.read_csv('data/movie_genres.csv')
movie_genres.rename(columns = {'name':'genre'}, inplace = True)
movie_genres = movie_genres.reindex(columns = ['movie_id','genre'])
movie_genres

,movie_id,genre
0,862,Animation
1,862,Comedy
2,862,Family
3,8844,Adventure
4,8844,Fantasy
...,...,...
91101,439050,Family
91102,111109,Drama
91103,67758,Action
91104,67758,Drama


In [5]:
# Checking unique values
movie_genres.genre.unique()

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music',
       'Documentary', 'Western', 'TV Movie', 'Carousel Productions',
       'Vision View Entertainment', 'Telescene Film Group Productions',
       'Aniplex', 'GoHands', 'BROSTA TV',
       'Mardock Scramble Production Committee', 'Sentai Filmworks',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel'],
      dtype=object)

In [6]:
# Filtering irrelevant values
movie_genres = movie_genres[~movie_genres['genre'].isin(['Carousel Productions',
       'Vision View Entertainment', 'Telescene Film Group Productions',
       'Aniplex', 'GoHands', 'BROSTA TV',
       'Mardock Scramble Production Committee', 'Sentai Filmworks',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel'])]

In [7]:
# Checking
movie_genres.genre.unique()

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music',
       'Documentary', 'Western', 'TV Movie'], dtype=object)

In [8]:
#Updating movie_genres csv file
movie_genres.to_csv('data/movie_genres.csv', index=False)

## Movie languages dataframe

In [9]:
movie_languages = pd.read_csv('data/movie_languages.csv')
movie_languages.rename(columns = {'name':'language','iso_639_1':'abbrevation'}, inplace = True)
movie_languages = movie_languages.reindex(columns = ['movie_id','language','abbrevation'])
movie_languages

,movie_id,language,abbrevation
0,862,English,en
1,8844,English,en
2,8844,Français,fr
3,15602,English,en
4,31357,English,en
...,...,...,...
53295,30840,English,en
53296,439050,فارسی,fa
53297,111109,NaN,tl
53298,67758,English,en


In [10]:
# Checking unique values
movie_languages.language.unique()

array(['English', 'Français', 'Español', 'Deutsch', 'Pусский', 'Latin',
       'Nederlands', '广州话 / 廣州話', '普通话', 'Magyar', 'shqip', 'Italiano',
       '한국어/조선말', 'فارسی', 'Dansk', nan, '日本語', 'العربية', 'Hrvatski',
       'Bosanski', 'Română', 'Bahasa indonesia', 'Bahasa melayu',
       'svenska', 'עִבְרִית', 'Český', 'Polski', 'Gaeilge', 'Norsk',
       'Slovenčina', 'Tiếng Việt', 'Português', 'हिन्दी', 'Català',
       'Íslenska', 'Afrikaans', 'Srpski', 'বাংলা', 'Wolof', 'Cymraeg',
       'ภาษาไทย', 'Latviešu', 'Kiswahili', 'български език', 'ελληνικά',
       'Türkçe', 'suomi', 'Esperanto', 'Український', 'ქართული', 'Bokmål',
       'No Language', 'euskera', 'Azərbaycan', 'Malti', 'اردو', 'isiZulu',
       'Bamanankan', 'پښتو', 'Somali', 'ਪੰਜਾਬੀ', 'беларуская мова',
       'தமிழ்', 'Galego', 'Kinyarwanda', 'қазақ', 'Eesti',
       'Lietuvi\x9akai', 'Slovenščina', 'తెలుగు', 'Fulfulde', '??????',
       '?????', 'ozbek', 'Hausa'], dtype=object)

In [11]:
# Filtering irrelevant values
movie_languages = movie_languages[~movie_languages['language'].isin(['??????','?????'])]

In [12]:
# Checking
movie_languages.language.unique()

array(['English', 'Français', 'Español', 'Deutsch', 'Pусский', 'Latin',
       'Nederlands', '广州话 / 廣州話', '普通话', 'Magyar', 'shqip', 'Italiano',
       '한국어/조선말', 'فارسی', 'Dansk', nan, '日本語', 'العربية', 'Hrvatski',
       'Bosanski', 'Română', 'Bahasa indonesia', 'Bahasa melayu',
       'svenska', 'עִבְרִית', 'Český', 'Polski', 'Gaeilge', 'Norsk',
       'Slovenčina', 'Tiếng Việt', 'Português', 'हिन्दी', 'Català',
       'Íslenska', 'Afrikaans', 'Srpski', 'বাংলা', 'Wolof', 'Cymraeg',
       'ภาษาไทย', 'Latviešu', 'Kiswahili', 'български език', 'ελληνικά',
       'Türkçe', 'suomi', 'Esperanto', 'Український', 'ქართული', 'Bokmål',
       'No Language', 'euskera', 'Azərbaycan', 'Malti', 'اردو', 'isiZulu',
       'Bamanankan', 'پښتو', 'Somali', 'ਪੰਜਾਬੀ', 'беларуская мова',
       'தமிழ்', 'Galego', 'Kinyarwanda', 'қазақ', 'Eesti',
       'Lietuvi\x9akai', 'Slovenščina', 'తెలుగు', 'Fulfulde', 'ozbek',
       'Hausa'], dtype=object)

In [13]:
#Updating movie_genres csv file
movie_languages.to_csv('data/movie_languages.csv', index=False)

## Production Companies dataframe

In [14]:
production_companies = pd.read_csv('data/production_companies.csv')
production_companies = production_companies.reindex(columns = ['movie_id','id','name'])
production_companies.to_csv('data/production_companies.csv', index=False)
production_companies

,movie_id,id,name
0,862,3,Pixar Animation Studios
1,8844,559,TriStar Pictures
2,8844,2550,Teitler Film
3,8844,10201,Interscope Communications
4,15602,6194,Warner Bros.
...,...,...,...
70540,30840,16323,20th Century Fox Television
70541,30840,38978,CanWest Global Communications
70542,111109,19653,Sine Olivia
70543,67758,6165,American World Pictures


## Production Countries dataframe

In [15]:
production_countries = pd.read_csv('data/production_countries.csv')
production_countries.rename(columns = {'name':'country', 'iso_3166_1':'abbrevation'}, inplace = True)
production_countries = production_countries.reindex(columns = ['movie_id','country','abbrevation'])
production_countries.to_csv('data/production_countries.csv', index=False)
production_countries

,movie_id,country,abbrevation
0,862,United States of America,US
1,8844,United States of America,US
2,15602,United States of America,US
3,31357,United States of America,US
4,11862,United States of America,US
...,...,...,...
49418,439050,Iran,IR
49419,111109,Philippines,PH
49420,67758,United States of America,US
49421,227506,Russia,RU


## Movies metadata dataframe

In [16]:
pd.set_option('display.max_columns', None)
#movies_metadata_raw

In [17]:
# Dropping duplicates
movies_metadata_raw.drop_duplicates(inplace=True, ignore_index=True)

# More than 90% values in 'status' column is 'Released',so we can drop this column
movies_metadata_raw = movies_metadata_raw[movies_metadata_raw['status'].isin(['Released'])]
movies_metadata_raw.drop('status', axis=1, inplace=True) 

#repeated_titles = pd.DataFrame(movies_metadata.title.value_counts().reset_index(name="count").query("count > 1")["index"])
#repeated_titles_list = repeated_titles.values.tolist()

# Drop irrelevant columns
movies_metadata_raw.drop(columns = ['adult','homepage','poster_path','video','tagline','original_title'],axis = 1,inplace = True)
movies_metadata_raw

#'release_year' column
movies_metadata_raw['release_year'] = movies_metadata_raw['release_date'].str[:4]
movies_metadata_raw['release_year'] = movies_metadata_raw['release_year'].fillna(0)
movies_metadata_raw.drop('release_date', axis=1, inplace=True)

# Popularity
movies_metadata_raw['popularity'][movies_metadata_raw.popularity == 'Beware Of Frost Bites'] = 0

In [18]:
# Checking types
movies_metadata_raw.dtypes

budget                object
id                    object
imdb_id               object
original_language     object
overview              object
popularity            object
revenue              float64
runtime              float64
title                 object
vote_average         float64
vote_count           float64
release_year          object
dtype: object

In [19]:
# Converting 'popularity' and 'budget'columns to numeric
movies_metadata_raw['popularity'] = pd.to_numeric(movies_metadata_raw.popularity, errors='coerce')
movies_metadata_raw['budget'] = pd.to_numeric(movies_metadata_raw.budget, errors='coerce')

In [20]:
# Checking NaNs
movies_metadata_raw.isna().sum()

# Only 'runtime' has Nulls(251)
movies_metadata_raw.fillna(0,inplace = True)

In [21]:
# Checking
movies_metadata_raw.isna().sum()

budget               0
id                   0
imdb_id              0
original_language    0
overview             0
popularity           0
revenue              0
runtime              0
title                0
vote_average         0
vote_count           0
release_year         0
dtype: int64

In [22]:
# Rearranging columns
movies_metadata_raw = movies_metadata_raw[['id', 'title', 'popularity','budget','revenue','runtime','vote_average', 'vote_count', 'overview']]

# Send cleaned data to new csv file
movies_metadata_raw.to_csv('data/movies_metadata_1.csv', index=False)
movies = pd.read_csv('data/movies_metadata_1.csv')
movies

,id,title,popularity,budget,revenue,runtime,vote_average,vote_count,overview
0,862,Toy Story,21.946943,30000000,373554033.0,81.0,7.7,5415.0,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,17.015539,65000000,262797249.0,104.0,6.9,2413.0,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,11.712900,0,0.0,101.0,6.5,92.0,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,3.859495,16000000,81452156.0,127.0,6.1,34.0,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,8.387519,0,76578911.0,106.0,5.7,173.0,Just when George Banks has recovered from his ...
...,...,...,...,...,...,...,...,...,...
44997,439050,Subdue,0.072051,0,0.0,90.0,4.0,1.0,Rising and falling between a man and woman.
44998,111109,Century of Birthing,0.178241,0,0.0,360.0,9.0,3.0,An artist struggles to finish his work while a...
44999,67758,Betrayal,0.903007,0,0.0,90.0,3.8,6.0,"When one of her hits goes wrong, a professiona..."
45000,227506,Satan Triumphant,0.003503,0,0.0,87.0,0.0,0.0,"In a small town live two brothers, one a minis..."


## Credits dataframe

In [23]:
credits = pd.read_csv("data/credits.csv")
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


### 'Cast' column

In [24]:
credits.cast[0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [25]:
# Applying cleaning function to the 'cast' column
cast = to_df(credits, 'cast')
cast

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,862
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,862
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,862
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,862
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,862
...,...,...,...,...,...,...,...,...,...
562469,2,,52fe4ea59251416c7515d7d5,2,544742,Iwan Mosschuchin,0,None,227506
562470,3,,52fe4ea59251416c7515d7d9,1,1090923,Nathalie Lissenko,1,None,227506
562471,4,,52fe4ea59251416c7515d7dd,2,1136422,Pavel Pavlov,2,None,227506
562472,5,,52fe4ea59251416c7515d7e1,0,1261758,Aleksandr Chabrov,3,None,227506


In [26]:
cast.drop(columns = ['id','credit_id','profile_path','gender'],inplace = True)
cast = cast[~cast['character'].isin([''])]
cast

,cast_id,character,name,order,movie_id
0,14,Woody (voice),Tom Hanks,0,862
1,15,Buzz Lightyear (voice),Tim Allen,1,862
2,16,Mr. Potato Head (voice),Don Rickles,2,862
3,17,Slinky Dog (voice),Jim Varney,3,862
4,18,Rex (voice),Wallace Shawn,4,862
...,...,...,...,...,...
562464,16,Sammy Benetto,Joe Sabatino,10,67758
562465,17,Steve,Kiko Ellsworth,11,67758
562466,18,Fred,Don Swayze,12,67758
562467,19,"Artie, Hitman #1",Peter Dobson,13,67758


In [27]:
# Creating 'cast' csv file
cast.to_csv('data/cast.csv', index=False)

### 'Crew' column

In [28]:
credits.crew[0][:1000]

"[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, 'job': 'Director', 'name': 'John Lasseter', 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}, {'credit_id': '52fe4284c3a36847f8024f4f', 'department': 'Writing', 'gender': 2, 'id': 12891, 'job': 'Screenplay', 'name': 'Joss Whedon', 'profile_path': '/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg'}, {'credit_id': '52fe4284c3a36847f8024f55', 'department': 'Writing', 'gender': 2, 'id': 7, 'job': 'Screenplay', 'name': 'Andrew Stanton', 'profile_path': '/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg'}, {'credit_id': '52fe4284c3a36847f8024f5b', 'department': 'Writing', 'gender': 2, 'id': 12892, 'job': 'Screenplay', 'name': 'Joel Cohen', 'profile_path': '/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg'}, {'credit_id': '52fe4284c3a36847f8024f61', 'department': 'Writing', 'gender': 0, 'id': 12893, 'job': 'Screenplay', 'name': 'Alec Sokolow', 'profile_path': '/v79vlRYi94BZUQnkkyznbGUZLjT.jpg'}, {'credit_id': '52fe4284c3a36847f8024f67', 'depar

In [29]:
# Applying cleaning function to the 'crew' column
crew = to_df(credits, 'crew')
crew

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg,862
1,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg,862
2,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg,862
3,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg,862
4,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg,862
...,...,...,...,...,...,...,...,...
464309,52fe4776c3a368484e0c8399,Sound,0,549356,Original Music Composer,Richard McHugh,None,67758
464310,52fe4776c3a368484e0c839f,Camera,2,58818,Director of Photography,João Fernandes,None,67758
464311,533bccebc3a36844cf0011a7,Directing,0,1085341,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg,227506
464312,58ebbc26925141281908aa0a,Production,2,1195656,Producer,Joseph N. Ermolieff,None,227506


In [30]:
crew.drop(columns = ['id','credit_id','profile_path','gender'],inplace = True)
crew

,department,job,name,movie_id
0,Directing,Director,John Lasseter,862
1,Writing,Screenplay,Joss Whedon,862
2,Writing,Screenplay,Andrew Stanton,862
3,Writing,Screenplay,Joel Cohen,862
4,Writing,Screenplay,Alec Sokolow,862
...,...,...,...,...
464309,Sound,Original Music Composer,Richard McHugh,67758
464310,Camera,Director of Photography,João Fernandes,67758
464311,Directing,Director,Yakov Protazanov,227506
464312,Production,Producer,Joseph N. Ermolieff,227506


In [31]:
# Creating 'crew' csv file
crew.to_csv('data/crew.csv', index=False)

## Keywords Dataframe

In [33]:
# Reading data
keywords = pd.read_csv('data/keywords_raw.csv.zip')
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [34]:
#Applying cleaning function
keywords = to_df(keywords,'keywords')
keywords.rename(columns = {'name':'keyword'})

,id,keyword,movie_id
0,931,jealousy,862
1,4290,toy,862
2,5202,boy,862
3,6054,friendship,862
4,9713,friends,862
...,...,...,...
158675,11800,mockumentary,289923
158676,10703,tragic love,439050
158677,2679,artist,111109
158678,14531,play,111109


In [35]:
#Extracting as csv file
keywords.to_csv('data/keywords.csv', index=False)

In [36]:
# Filtering keywords that occur less than 50 times
kw_counts = keywords.name.value_counts()
popular_keywords = kw_counts[kw_counts > 50].index

keywords = keywords[keywords.name.isin(popular_keywords)]

In [37]:
keywords.shape

(68221, 3)